In [2]:
import requests

# Fetch the JSON data
url = "https://meta.wikimedia.org/w/api.php?action=sitematrix&format=json"
response = requests.get(url)
data = response.json()

# Extract and clean URLs
domains = []

sitematrix = data.get("sitematrix", {})
for key, val in sitematrix.items():
    if key in ("count", "specials"):
        continue
    if isinstance(val, dict):
        sites = val.get("site", [])
        for site in sites:
            site_url = site.get("url")
            if site_url:
                cleaned_url = site_url.replace("https://", "")
                domains.append(cleaned_url)

# Example: print or use the list
print(domains)


['aa.wikipedia.org', 'aa.wiktionary.org', 'aa.wikibooks.org', 'ab.wikipedia.org', 'ab.wiktionary.org', 'ace.wikipedia.org', 'ady.wikipedia.org', 'af.wikipedia.org', 'af.wiktionary.org', 'af.wikibooks.org', 'af.wikiquote.org', 'ak.wikipedia.org', 'ak.wiktionary.org', 'ak.wikibooks.org', 'alt.wikipedia.org', 'am.wikipedia.org', 'am.wiktionary.org', 'am.wikiquote.org', 'ami.wikipedia.org', 'an.wikipedia.org', 'an.wiktionary.org', 'ang.wikipedia.org', 'ang.wiktionary.org', 'ang.wikibooks.org', 'ang.wikiquote.org', 'ang.wikisource.org', 'ann.wikipedia.org', 'anp.wikipedia.org', 'ar.wikipedia.org', 'ar.wiktionary.org', 'ar.wikibooks.org', 'ar.wikinews.org', 'ar.wikiquote.org', 'ar.wikisource.org', 'ar.wikiversity.org', 'arc.wikipedia.org', 'ary.wikipedia.org', 'arz.wikipedia.org', 'as.wikipedia.org', 'as.wiktionary.org', 'as.wikibooks.org', 'as.wikiquote.org', 'as.wikisource.org', 'ast.wikipedia.org', 'ast.wiktionary.org', 'ast.wikibooks.org', 'ast.wikiquote.org', 'atj.wikipedia.org', 'av.wi

In [ ]:
import requests
import pandas as pd
import pymysql
import configparser

# --- Load Toolforge DB credentials from replica.my.cnf ---
cfg = configparser.ConfigParser()
cfg.read('/data/project/community-activity-alerts-system/replica.my.cnf')
user = cfg['client']['user']
password = cfg['client']['password']

# --- Config ---
project = "uz.wikipedia.org"
base_url = "https://wikimedia.org/api/rest_v1/metrics/edits/aggregate"
editor_type = "all-editor-types"
page_type = "content"
granularity = "monthly"
start = "20200101"
end = "20240101"

# --- Construct URL and fetch data ---
url = f"{base_url}/{project}/{editor_type}/{page_type}/{granularity}/{start}/{end}"
response = requests.get(url)
if response.status_code != 200:
    raise Exception(f"API Error: {response.status_code} - {response.text}")
data = response.json()
edit_counts = data["items"][0]["results"]

df = pd.DataFrame(edit_counts)
df['timestamp'] = pd.to_datetime(df['timestamp'], utc=True)
df['project'] = project
df.rename(columns={'edits': 'edit_count'}, inplace=True)

# --- Save DataFrame to Toolforge database ---
DB_NAME = 's56391__community_alerts'
DB_TABLE = 'edit_counts'

conn = pymysql.connect(
    host='tools.db.svc.wikimedia.cloud',
    user=user,
    password=password,
    database=DB_NAME,
    charset='utf8mb4',
    autocommit=True
)

cursor = conn.cursor()

create_table_sql = f'''
CREATE TABLE IF NOT EXISTS {DB_TABLE} (
    timestamp DATETIME,
    edit_count INT,
    project VARCHAR(255),
    PRIMARY KEY (timestamp, project)
)
'''
cursor.execute(create_table_sql)

for _, row in df.iterrows():
    insert_sql = f"""
    INSERT INTO {DB_TABLE} (timestamp, edit_count, project)
    VALUES (%s, %s, %s)
    ON DUPLICATE KEY UPDATE edit_count = VALUES(edit_count)
    """
    cursor.execute(insert_sql, (row['timestamp'].to_pydatetime(), int(row['edit_count']), row['project']))

cursor.close()
conn.close()

print('Data saved to Toolforge database.')
